Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [122]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
from random import choice

In [123]:
# function that prints sets in a human readable way
def print_solution(solution, sets):
    
    for i, is_set_covered in enumerate(solution):
        if is_set_covered:
            print('Set', i+1, ': ', end='')
            for element_taken in sets.toarray()[i]:
                if(element_taken): print('x', end='')
                else: print('-', end='')
            print()


In [124]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [125]:
# x = make_set_covering_problem(1000, 1000, .3)
# print("Element at row=42 and column=42:", x[42, 42])

In [126]:
x1 = make_set_covering_problem(100, 100, .3)

In [127]:
x2 = make_set_covering_problem(1000, 1000, .3)

In [128]:
x3 = make_set_covering_problem(5000, 5000, .3)

KeyboardInterrupt: 

In [ ]:
x4 = make_set_covering_problem(100, 100, .7)

In [ ]:
x5 = make_set_covering_problem(1000, 1000, .7)

In [ ]:
x6 = make_set_covering_problem(5000, 5000, .7)

In [ ]:
def fitness(state, sets):
    SETS = sets.toarray()
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(SETS[0]))]),
        )
    )
    return valid, sum(state)

In [ ]:
def tweak(state, PROBLEM_SIZE, valid):
    new_state = copy(state)
    if valid:
        taken = []
        for i, t in enumerate(state):
            if t:
                taken.append(i)
        index = choice(taken)
    else: index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

# Algorithm explaination

## Idea

The algorithms starts with a state that does not have any set taken and it has two consecutive phases:
1. Initially it tries to form a valid solution by taking random sets.
2. When a valid solution is found it tries to improve it by removing sets that are not needed to the solution.

## Parameters

The algorithm has the following parameters:
- **initial_state**: no set is taken
- **sets**: the list of all the sets, used to validate the solutions
- **max_improvement**: the maximum number of steps in the second phase
- **max_consecutive**: True if the max_improvement are considered consecutive, False if they are total
- **total_steps**: the maximum number of steps of the algorithm

In [ ]:
def single_state_solve(initial_state, sets, max_improvement=100, max_consecutive=False,total_steps=1_000):
    cycles_needed = 0
    current_state = copy(initial_state)
    valid, current_fitness = fitness(current_state, sets)
    number_of_sets = len(initial_state)
    prev_valid = valid
    failed_improvement = 0
    
    for _ in range(total_steps):
        if failed_improvement >= max_improvement:
            break
        cycles_needed += 1
        new_state = tweak(current_state, number_of_sets, valid)
        valid, new_fitness = fitness(new_state, sets)
        if valid and new_fitness < current_fitness:
            #print(f"Improvement at cycle: {cycles_needed}, ({valid}, {new_fitness})") 
            current_state = new_state
            current_fitness = new_fitness
            if max_consecutive: failed_improvement = 0
        elif valid and not prev_valid:
            #print(f"Found a valid solution at cycle: {cycles_needed}, ({valid}, {new_fitness})")
            current_state = new_state
            current_fitness = new_fitness
        elif not valid and prev_valid:
            failed_improvement += 1
            valid = prev_valid
            continue
        elif not valid and not prev_valid and new_fitness > current_fitness:
            #print(f"Still searching a valid solution. Current size {new_fitness}")
            current_state = new_state
            current_fitness = new_fitness
        prev_valid = valid

    return current_state, cycles_needed

In [137]:
initial_state_100 = [False for _ in range(100)]
initial_state_1000 = [False for _ in range(1000)]
initial_state_5000 = [False for _ in range(5000)]

max_improvement = 5
max_consecutive = True

In [138]:
solution1, cycles = single_state_solve(initial_state_100, x1, max_improvement, max_consecutive)
print(f"Solution has fitness: {fitness(solution1, x1)} in {cycles} cycles.")

Solution has fitness: (True, 9) in 44 cycles.


In [139]:
solution2, cycles = single_state_solve(initial_state_1000, x2, max_improvement, max_consecutive)
print(f"Solution has fitness: {fitness(solution2, x2)} in {cycles} cycles.")

Solution has fitness: (True, 15) in 39 cycles.


In [140]:
solution3, cycles = single_state_solve(initial_state_5000, x3, max_improvement, max_consecutive)
print(f"Solution has fitness: {fitness(solution3, x3)} in {cycles} cycles.")

Solution has fitness: (True, 20) in 27 cycles.


In [141]:
solution4, cycles = single_state_solve(initial_state_100, x4, max_improvement, max_consecutive)
print(f"Solution has fitness: {fitness(solution4, x4)} in {cycles} cycles.")

Solution has fitness: (True, 4) in 9 cycles.


In [142]:
solution5, cycles = single_state_solve(initial_state_1000, x5, max_improvement, max_consecutive)
print(f"Solution has fitness: {fitness(solution5, x5)} in {cycles} cycles.")

Solution has fitness: (True, 6) in 15 cycles.


In [143]:
solution6, cycles = single_state_solve(initial_state_5000, x6, max_improvement, max_consecutive)
print(f"Solution has fitness: {fitness(solution6, x6)} in {cycles} cycles.")

# Results

## Maximum failed improvements: 10

I set up the algorithm to have a maximum of 10 improvements after which it needed to stop.

### First run
| Num Points & Sets | Density | Solution Size | Number of Cycles |
| ----------------- | ------- | ------------- | ---------------- |
| 100               | .3      | 9             | 31               |
| 1000              | .3      | 16            | 32               |
| 5000              | .3      | 25            | 41               |
| 100               | .7      | 3             | 15               |
| 1000              | .7      | 6             | 16               |
| 5000              | .7      | 7             | 19               |

### Second run
| Num Points & Sets | Density | Solution Size | Number of Cycles |
| ----------------- | ------- | ------------- | ---------------- |
| 100               | .3      | 9             | 31               |
| 1000              | .3      | 18            | 39               |
| 5000              | .3      | 22            | 36               |
| 100               | .7      | 3             | 13               |
| 1000              | .7      | 5             | 21               |
| 5000              | .7      | 7             | 19               |

### Third run
| Num Points & Sets | Density | Solution Size | Number of Cycles |
| ----------------- | ------- | ------------- | ---------------- |
| 100               | .3      | 11             | 24               |
| 1000              | .3      | 18            | 39               |
| 5000              | .3      | 22            | 36               |
| 100               | .7      | 3             | 13               |
| 1000              | .7      | 5             | 21               |
| 5000              | .7      | 7             | 19               |

## Maximum number of consecutive failed improvements: 10

I added a reset of the number of improvements in case the algorithm find a better solution. In this way each a better solution is found, the algorithm tries 10 times to improve it and after that it stops

### Runs

Three runs have been made generating each time different sets and the results were the same for each run.

| Num Points & Sets | Density | Solution Size | Number of Cycles |
| ----------------- | ------- | ------------- | ---------------- |
| 100               | .3      | 11            | 24               |
| 1000              | .3      | 16            | 54               |
| 5000              | .3      | 19            | 47               |
| 100               | .7      | 4             | 17               |
| 1000              | .7      | 6             | 21               |
| 5000              | .7      | 8             | 18               |


## Maximum failed improvements: 5

Given that the challenge objective is to find the best solution with the fewest calls to the fitness functions, I decided to try to reduce the number of failed improvements to 5.

| Num Points & Sets | Density | Solution Size | Number of Cycles |
| ----------------- | ------- | ------------- | ---------------- |
| 100               | .3      | 11            | 38               |
| 1000              | .3      | 17            | 22               |
| 5000              | .3      | 18            | 33               |
| 100               | .7      | 3             | 10               |
| 1000              | .7      | 6             | 13               |
| 5000              | .7      | 7             | 14               |

## Maximum number of concecutives failed improvements: 5.

Same reasoning as before.

| Num Points & Sets | Density | Solution Size | Number of Cycles |
| ----------------- | ------- | ------------- | ---------------- |
| 100               | .3      | 11            | 19               |
| 1000              | .3      | 21            | 35               |
| 5000              | .3      | 20            | 39               |
| 100               | .7      | 4             | 9               |
| 1000              | .7      | 6             | 11               |
| 5000              | .7      | 7             | 18               |